In [ ]:
import boto3
import time

def lambda_handler(event, context):
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    workgroup_name = 'walmart-bie-workgroup'
    database_name = 'dev'
    iam_role = 'arn:aws:iam::748065983427:role/redshift-serverless-s3-execution-role'

    redshift = boto3.client('redshift-data')

    time.sleep(5)

    copy_sql = f"""
    COPY walmart_sales
    FROM 's3://{bucket}/{key}'
    IAM_ROLE '{iam_role}'
    FORMAT AS CSV
    IGNOREHEADER 1;
    """

    redshift.execute_statement(
        WorkgroupName=workgroup_name,
        Database=database_name,
        Sql=copy_sql
    )

    # Simple CloudWatch log entry
    print(f"COPY command submitted for: s3://{bucket}/{key}")

    return {
        'statusCode': 200,
        'body': f'Submitted COPY command for {key}'
    }
